In [ ]:
import os
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbedding
from langchain_groq import chatGroq
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage


In [2]:
load_dotenv()

True

In [4]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os. getenv("LANGCHAIN_PROJECT")

In [5]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os. environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os. environ["LANGCHAIN_API_KEY"]= LANGCHAIN_API_KEY
os. environ["LANGCHAIN_TRACING_V2"] = "true"
os. environ["LANGCHAIN_ENDPOINT"] = "https://api.smith. langchain.com"
os. environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro")

In [ ]:
"""
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
llm = ChatGroq(model_name='Gemma2-9b-It')

"""

### Simple AI Assistant

In [ ]:
while True:
    question = input("Type your question. if you want to quit the chat , write quit")
    if question.lower() != "quit":
        print(llm.invoke(question).content)

    else:
        print("Good bye till next time")
        break

In [ ]:
store = {}

In [ ]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()

    return store[session_id]

In [ ]:
config = {"configurable": {"session_id": "firstchat"}}

In [ ]:
model_with_memory = RunnableWithMessageHistory(llm, get_session_history)

In [ ]:
model_with_memory.invoke(("Hi, I'm Hamid"), config=config).content

In [ ]:
model_with_memory.invoke(("tell me what is my name"), config=config).content

### RAG with LCEL

In [ ]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser


In [ ]:
# reading txt files from source directroy
loader = DirectoryLoader("../data", glob="./*.txt", loader_cls=TextLoader)

docs = loader.load()

In [ ]:
# Creating Chunks using RecursiveCharactertextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)

new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

In [ ]:
### BGE Embddings

'''from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs ='normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
model_name=model_name,
model_kwargs=model_kwargs,
encode_kwargs=encode_kwargs,
'''

In [ ]:
# Creating retriever using vector db
db = Chroma.from_documents(new_docs, embedding=embeddings)
retriever = db.as_retriever(search_kwargs={"k":4})

In [ ]:
template = """

Answer the question based only on the following context:
{context}

Question: {question}

"""

prompt = PromptTemplate.from_template(template)

In [ ]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
question_1 = "What is llama3? can you highlight 3 important points?"
print(retrieval_chain.invoke(question_1))

### Start with Tools and Agents

In [ ]:
from langchain_community.tools import WikipediaQueryRun, YouTubeSearchTool
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults


In [ ]:
! pip install wikipedia youtube_search

In [ ]:
api_wrapper = WikipediaAPIWrapper()

tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
tool.name, tool.description

In [ ]:
tool.args

In [ ]:
print(tool.run({"query": "langchain"}))

In [ ]:
tool_2 = YouTubeSearchTool()

In [ ]:
tool_2.name

In [ ]:
tool_2.run("freecodecamp")

In [ ]:
tool_3 = TavilySearchResults()

In [ ]:
tool_3.invoke({"query": "What happend in the latest burning man floods?"})

In [ ]:
from langchain.agents import AgentType, load_tools, initialize_agent


In [ ]:
tool_4 = load_tools(["wikipedia"], llm=llm)

In [ ]:
agent = initialize_agent(tool_4, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("What is current GDP of USA?")